# Importing Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import math 
import sys

In [3]:
csv_files = glob("./zippedData/*.csv.gz")
csv_files

['./zippedData\\bom.movie_gross.csv.gz',
 './zippedData\\imdb.name.basics.csv.gz',
 './zippedData\\imdb.title.akas.csv.gz',
 './zippedData\\imdb.title.basics.csv.gz',
 './zippedData\\imdb.title.crew.csv.gz',
 './zippedData\\imdb.title.principals.csv.gz',
 './zippedData\\imdb.title.ratings.csv.gz',
 './zippedData\\tmdb.movies.csv.gz',
 './zippedData\\tn.movie_budgets.csv.gz']

## Cleaning the file names

In [4]:
csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_").replace("_gz","") 
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df

In [5]:
csv_files_dict.keys()

dict_keys(['bom_movie_gross', 'imdb_name_basics', 'imdb_title_akas', 'imdb_title_basics', 'imdb_title_crew', 'imdb_title_principals', 'imdb_title_ratings', 'tmdb_movies', 'tn_movie_budgets'])

In [6]:
bom_movie_gross = csv_files_dict['bom_movie_gross']
imdb_name_basics = csv_files_dict['imdb_name_basics']
imdb_title_akas = csv_files_dict['imdb_title_akas']
imdb_title_basics = csv_files_dict['imdb_title_basics']
imdb_title_crew = csv_files_dict['imdb_title_crew']
imdb_title_principals = csv_files_dict['imdb_title_principals']
imdb_title_ratings = csv_files_dict['imdb_title_ratings']
tmdb_movies = csv_files_dict['tmdb_movies']
movie_budgets = csv_files_dict['tn_movie_budgets']

# Cleaning each database

## Box Office Mojo

In [7]:
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3387 entries, Toy Story 3 to An Actor Prepares
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   studio          3382 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   object 
 3   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 132.3+ KB


In [8]:
bom_movie_gross
#drop foreign_gross,use studio if need

,studio,domestic_gross,foreign_gross,year
title,,,,
Toy Story 3,BV,415000000.0,652000000,2010
Alice in Wonderland (2010),BV,334200000.0,691300000,2010
Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
Inception,WB,292600000.0,535700000,2010
Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...
The Quake,Magn.,6200.0,NaN,2018
Edward II (2018 re-release),FM,4800.0,NaN,2018
El Pacto,Sony,2500.0,NaN,2018


In [9]:
print('Percentage of Null foreign_gross values:', len(bom_movie_gross[bom_movie_gross.foreign_gross.isna()])/ len(bom_movie_gross))

Percentage of Null foreign_gross values: 0.3985828166519043


We are going to remove the foreign_gross column as 40% of values are null

In [10]:
bom_movie_gross = bom_movie_gross.drop(['foreign_gross'], axis = 1)

In [11]:
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3387 entries, Toy Story 3 to An Actor Prepares
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   studio          3382 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 105.8+ KB


Still need to remove null values from other columns

In [12]:
bom_movie_gross = bom_movie_gross.dropna()
bom_movie_gross.isna().sum()

studio            0
domestic_gross    0
year              0
dtype: int64

In [13]:
bom_movie_gross.to_csv('bom_movie_gross_clean')

Null values removed! Data cleaned in bom_movie_gross_gz

## The Numbers: Movie Budgets

In [14]:
movie_budgets.head() 

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [15]:
movie_budgets.sort_values(by='domestic_gross')

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
82,"Oct 26, 2012",Mientras duermes,"$5,000,000",$0,"$9,109,597"
9,"Nov 4, 2005",Wal-Mart: The High Cost of Low Price,"$1,500,000",$0,"$58,692"
8,"Jul 20, 2018",Teefa in Trouble,"$1,500,000",$0,"$98,806"
7,"Oct 17, 2014",Housebound,"$1,500,000",$0,"$236,863"
58,"Dec 31, 2008",Bathory,"$15,000,000",$0,"$3,436,763"
...,...,...,...,...,...
8,"Jun 13, 1997",Hercules,"$70,000,000","$99,112,101","$250,700,000"
74,"Jun 4, 1999",Desert Blue,"$5,000,000","$99,147","$99,147"
43,"Aug 3, 2018",Christopher Robin,"$75,000,000","$99,215,042","$197,504,758"


In [16]:
movie_budgets.worldwide_gross.value_counts(normalize= True)

$0             0.063473
$8,000,000     0.001557
$7,000,000     0.001038
$2,000,000     0.001038
$5,000,000     0.000692
                 ...   
$10,182,050    0.000173
$36,720,752    0.000173
$11,831,131    0.000173
$500,296       0.000173
$12,309,980    0.000173
Name: worldwide_gross, Length: 5356, dtype: float64

In [17]:
movie_budgets.production_budget.value_counts(normalize= True)

$20,000,000     0.039952
$10,000,000     0.036666
$30,000,000     0.030612
$15,000,000     0.029920
$25,000,000     0.029575
                  ...   
$207,000,000    0.000173
$110,000        0.000173
$23,000         0.000173
$60,720,000     0.000173
$34,800,000     0.000173
Name: production_budget, Length: 509, dtype: float64

In [18]:
movie_budgets.domestic_gross.value_counts(normalize= True)

$0              0.094777
$8,000,000      0.001557
$7,000,000      0.001211
$2,000,000      0.001211
$10,000,000     0.001038
                  ...   
$5,969,553      0.000173
$14,915,773     0.000173
$1,196,321      0.000173
$100,468,793    0.000173
$3,395,581      0.000173
Name: domestic_gross, Length: 5164, dtype: float64

In [19]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 271.0+ KB


In [20]:
def convert_amt_to_int(df, col):
    df[col] = df[col].str.replace("$", "").str.replace(",", "").astype('int64')
    return df

In [21]:
money_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in money_cols:
    movie_budgets = convert_amt_to_int(movie_budgets, col)

In [22]:
movie_budgets = movie_budgets[movie_budgets['domestic_gross'] > 0]

In [23]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5234 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5234 non-null   object
 1   movie              5234 non-null   object
 2   production_budget  5234 non-null   int64 
 3   domestic_gross     5234 non-null   int64 
 4   worldwide_gross    5234 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 245.3+ KB


In [24]:
movie_budgets.to_csv('movie_budgets_clean.csv')

## IMDB "Name Basics"

In [25]:
imdb_name_basics
#contains actors, directors etc

,primary_name,birth_year,death_year,primary_profession,known_for_titles
nconst,,,,,
nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"
...,...,...,...,...,...
nm9990381,Susan Grobes,NaN,NaN,actress,NaN
nm9990690,Joo Yeon So,NaN,NaN,actress,"tt9090932,tt8737130"
nm9991320,Madeline Smith,NaN,NaN,actress,"tt8734436,tt9615610"


In [26]:
line_1 =imdb_name_basics.iloc[0]

In [27]:
def print_line(row):
    list_of_rows=[]
    for title_id in row['known_for_titles']:
        list_of_rows.append((row[['nconst', 'primary_profession', 'known_for_title']]))
        


In [28]:
imdb_name_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606648 entries, nm0061671 to nm9993380
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   primary_name        606648 non-null  object 
 1   birth_year          82736 non-null   float64
 2   death_year          6783 non-null    float64
 3   primary_profession  555308 non-null  object 
 4   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(3)
memory usage: 27.8+ MB


In [29]:
print('Percentage of Null birth_year Values:', len(imdb_name_basics[imdb_name_basics.birth_year.isna()])/ len(imdb_name_basics))

Percentage of Null birth_year Values: 0.8636177816460286


In [30]:
print('Percentage of Null death_year Values:', len(imdb_name_basics[imdb_name_basics.death_year.isna()])/ len(imdb_name_basics))

Percentage of Null death_year Values: 0.9888188867349764


In [31]:
imdb_name_basics = imdb_name_basics.drop('birth_year', axis = 1)

In [32]:
imdb_name_basics = imdb_name_basics.drop('death_year', axis = 1)

In [33]:
imdb_name_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606648 entries, nm0061671 to nm9993380
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   primary_name        606648 non-null  object
 1   primary_profession  555308 non-null  object
 2   known_for_titles    576444 non-null  object
dtypes: object(3)
memory usage: 18.5+ MB


In [34]:
imdb_name_basics = imdb_name_basics.dropna()
imdb_name_basics.isna().sum()

primary_name          0
primary_profession    0
known_for_titles      0
dtype: int64

In [35]:
imdb_name_basics.to_csv('imdb_name_basics_clean.csv')

## IMDB "Title Akas"

In [36]:
imdb_title_akas.sort_values(by='region')

,ordering,title,region,language,types,attributes,is_original_title
title_id,,,,,,,
tt3156000,3,Nick,AD,NaN,NaN,NaN,0.0
tt6079382,1,Impacto,AD,NaN,NaN,NaN,0.0
tt5162282,1,73',AD,NaN,NaN,NaN,0.0
tt1811329,2,"Teta, Alf Marra",AE,NaN,NaN,NaN,0.0
tt7480896,1,Desert Dream,AE,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...
tt9723084,2,Anderswo. Allein in Afrika,NaN,NaN,original,NaN,1.0
tt9726638,2,Monkey King: The Volcano,NaN,NaN,original,NaN,1.0
tt9755806,3,Big Shark,NaN,NaN,original,NaN,1.0


In [37]:
imdb_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331703 entries, tt0369610 to tt9880178
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ordering           331703 non-null  int64  
 1   title              331703 non-null  object 
 2   region             278410 non-null  object 
 3   language           41715 non-null   object 
 4   types              168447 non-null  object 
 5   attributes         14925 non-null   object 
 6   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 20.2+ MB


In [38]:
print('Percentage of Null death_year Values:', len(imdb_title_akas[imdb_title_akas.language.isna()])/ len(imdb_title_akas))

Percentage of Null death_year Values: 0.87423990738703


In [39]:
print('Percentage of Null death_year Values:', len(imdb_title_akas[imdb_title_akas.attributes.isna()])/ len(imdb_title_akas))

Percentage of Null death_year Values: 0.9550049291082685


In [40]:
print('Percentage of Null death_year Values:', len(imdb_title_akas[imdb_title_akas.types.isna()])/ len(imdb_title_akas))

Percentage of Null death_year Values: 0.49217522904526034


In [41]:
imdb_title_akas = imdb_title_akas.drop('types', axis = 1)

In [42]:
imdb_title_akas = imdb_title_akas.drop('attributes', axis = 1)

In [43]:
imdb_title_akas = imdb_title_akas.drop('language', axis = 1)

In [44]:
imdb_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331703 entries, tt0369610 to tt9880178
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ordering           331703 non-null  int64  
 1   title              331703 non-null  object 
 2   region             278410 non-null  object 
 3   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 12.7+ MB


In [45]:
imdb_title_akas['is_original_title'] = imdb_title_akas['is_original_title'].fillna(value = 0)

In [46]:
imdb_title_akas['region'] = imdb_title_akas['region'].fillna(value= 'no region')

In [47]:
imdb_title_akas.to_csv('imdb_title_akas_clean.csv')

## IMDB "Title Crew"

In [48]:
imdb_title_crew

,directors,writers
tconst,,
tt0285252,nm0899854,nm0899854
tt0438973,NaN,"nm0175726,nm1802864"
tt0462036,nm1940585,nm1940585
tt0835418,nm0151540,"nm0310087,nm0841532"
tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943
...,...,...
tt8999974,nm10122357,nm10122357
tt9001390,nm6711477,nm6711477
tt9001494,"nm10123242,nm10123248",NaN


In [49]:
imdb_title_crew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0285252 to tt9010172
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   directors  140417 non-null  object
 1   writers    110261 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [50]:
imdb_title_crew.isna().sum()

directors     5727
writers      35883
dtype: int64

In [51]:
imdb_title_crew['writers'] = imdb_title_crew['writers'].fillna(value= 'unknown')
imdb_title_crew['directors'] = imdb_title_crew['directors'].fillna(value= 'unknown')

In [52]:
imdb_title_crew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0285252 to tt9010172
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   directors  146144 non-null  object
 1   writers    146144 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [53]:
imdb_title_crew.to_csv('imdb_title_crew_clean.csv')

## IMDB "Title Principals"

In [54]:
imdb_title_principals.head()

,ordering,nconst,category,job,characters
tconst,,,,,
tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
tt0111414,2,nm0398271,director,NaN,NaN
tt0111414,3,nm3739909,producer,producer,NaN
tt0323808,10,nm0059247,editor,NaN,NaN
tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [55]:
imdb_title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028186 entries, tt0111414 to tt9692684
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ordering    1028186 non-null  int64 
 1   nconst      1028186 non-null  object
 2   category    1028186 non-null  object
 3   job         177684 non-null   object
 4   characters  393360 non-null   object
dtypes: int64(1), object(4)
memory usage: 47.1+ MB


In [56]:
imdb_title_principals = imdb_title_principals.drop('job', axis = 1)
# Dropped job because it was redundent of catagory

In [57]:
imdb_title_principals['characters'] = imdb_title_principals['characters'].fillna(value= 'not applicable')

In [58]:
imdb_title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028186 entries, tt0111414 to tt9692684
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ordering    1028186 non-null  int64 
 1   nconst      1028186 non-null  object
 2   category    1028186 non-null  object
 3   characters  1028186 non-null  object
dtypes: int64(1), object(3)
memory usage: 39.2+ MB


In [59]:
imdb_title_principals.to_csv('imdb_title_principals_clean.csv')

## IMDB "Title Ratings"

In [60]:
imdb_title_ratings

,averagerating,numvotes
tconst,,
tt10356526,8.3,31
tt10384606,8.9,559
tt1042974,6.4,20
tt1043726,4.2,50352
tt1060240,6.5,21
...,...,...
tt9805820,8.1,25
tt9844256,7.5,24
tt9851050,4.7,14


In [61]:
imdb_title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73856 entries, tt10356526 to tt9894098
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   averagerating  73856 non-null  float64
 1   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 1.7+ MB


In [62]:
imdb_title_ratings.averagerating.value_counts(normalize= False)

7.0     2262
6.6     2251
7.2     2249
6.8     2239
6.5     2221
        ... 
9.6       18
10.0      16
9.8       15
9.7       12
9.9        5
Name: averagerating, Length: 91, dtype: int64

In [63]:
imdb_title_ratings.numvotes.value_counts(normalize= True)

6        0.038927
5        0.036544
7        0.033525
8        0.029341
9        0.026118
           ...   
18286    0.000014
16289    0.000014
1958     0.000014
4007     0.000014
4098     0.000014
Name: numvotes, Length: 7349, dtype: float64

In [64]:
imdb_title_ratings.to_csv('imdb_title_ratings_clean.csv')

## The Movie DB "Movies"

In [65]:
tmdb_movies

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [66]:
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [67]:
list_of_values= []
def value_count(df):
    for header in df:
         list_of_values.append(df[header].value_counts(normalize= True))
    

In [68]:
value_count(tmdb_movies)
list_of_values

[[99]                    0.139533
 []                      0.093487
 [18]                    0.085530
 [35]                    0.062601
 [27]                    0.043180
                           ...   
 [35, 80, 14, 27, 53]    0.000038
 [14, 27, 10749]         0.000038
 [18, 35, 16, 10751]     0.000038
 [878, 12, 14]           0.000038
 [16, 12, 10749]         0.000038
 Name: genre_ids, Length: 2477, dtype: float64,
 292086    0.000113
 463839    0.000113
 11976     0.000113
 391872    0.000113
 416572    0.000113
             ...   
 356987    0.000038
 350846    0.000038
 479871    0.000038
 500353    0.000038
 524288    0.000038
 Name: id, Length: 25497, dtype: float64,
 en    0.878342
 fr    0.019120
 es    0.017159
 ru    0.011238
 ja    0.009994
         ...   
 sw    0.000038
 mi    0.000038
 yi    0.000038
 hy    0.000038
 ps    0.000038
 Name: original_language, Length: 76, dtype: float64,
 Eden                                    0.000264
 Home                               

In [69]:
tmdb_movies = tmdb_movies[tmdb_movies['vote_count'] > 10]

In [70]:
tmdb_movies.to_csv('tmdb_movies_clean.csv')

## Rotten Tomatoes "Movie"

In [71]:
rt_movie = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter = '\t', encoding = 'unicode_escape')

In [72]:
rt_movie.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [73]:
rt_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [74]:
rt_movie = rt_movie.drop(['currency'], axis = 1)
rt_movie = rt_movie.drop(['box_office'], axis = 1)
rt_movie = rt_movie.drop(['dvd_date'], axis = 1)
rt_movie = rt_movie.drop(['runtime'], axis = 1)
rt_movie = rt_movie.drop(['studio'], axis = 1)
rt_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
dtypes: int64(1), object(6)
memory usage: 85.4+ KB


In [75]:
rt_movie.to_csv('rt_movie_clean.csv')

In [76]:
movie_budgets.head() 

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [77]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5234 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5234 non-null   object
 1   movie              5234 non-null   object
 2   production_budget  5234 non-null   int64 
 3   domestic_gross     5234 non-null   int64 
 4   worldwide_gross    5234 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 245.3+ KB


## Franchise Databases

In [78]:
franchises = pd.read_csv('franchises_clean.csv')

In [79]:
franchises

,Rank,Franchise,Total Gross,Movie Count,Average Gross
0,1,Marvel Cinematic Universe,"8,525",23,371
1,2,Star Wars,"4,221",11,384
2,3,Disney Live Action Reimaginings,"3,220",14,230
3,4,Wizarding World,"2,785",10,253
4,5,Avengers,"2,619",4,655
...,...,...,...,...,...
62,63,The Secret Life of Pets,525,2,263
63,64,"Monsters, Inc.",524,2,262
64,65,Kung Fu Panda,524,3,175
65,66,Rush Hour,508,3,169


In [80]:
franchises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             67 non-null     int64 
 1   Franchise        67 non-null     object
 2    Total Gross     67 non-null     object
 3    Movie Count     67 non-null     int64 
 4    Average Gross   67 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.7+ KB


In [81]:
franchises = franchises.set_index('Rank')

In [82]:
franchises.to_csv('franchises_clean.csv')

In [83]:
extfranchises = pd.read_csv('extfranchises_clean.csv', encoding='cp1252')

In [84]:
extfranchises

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Franchise,#movies,domesticboxoffice,infl_adj_box_office,worldwideboxoffice,firstyear,last year,#years
0,0,0,0,0,0,Marvel Cinematic Universe,31,8545426433,9102945801,22576311289,2008,2022,14
1,1,1,1,1,1,Star Wars,15,5079119579,8039797255,10323355075,1977,2026,49
2,2,2,2,2,2,Harry Potter,12,2786938291,3638704111,9195390708,2001,2021,20
3,3,3,3,3,3,Batman,22,2786778902,3949355696,6049670921,1989,2021,32
4,4,4,4,4,4,Spider-Man,11,2711115724,3292639145,7219770479,2002,2019,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,673,673,673,673,673,Django,2,19391,22137,19391,1966,2020,54
674,674,674,674,674,674,St. Trinian's,2,15000,18220,29830239,2009,2009,
675,675,675,675,675,675,The Wedding Party,2,11140,11140,73115,2017,2017,
676,676,676,676,676,676,Donald Strachey,2,4269,5402,4269,2008,2008,


In [85]:
extfranchises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           678 non-null    int64 
 1   Unnamed: 0.1         678 non-null    int64 
 2   Unnamed: 0.1.1       678 non-null    int64 
 3   Unnamed: 0.1.1.1     678 non-null    int64 
 4   Unnamed: 0.1.1.1.1   678 non-null    int64 
 5   Franchise            678 non-null    object
 6   #movies              678 non-null    int64 
 7   domesticboxoffice    678 non-null    int64 
 8   infl_adj_box_office  678 non-null    int64 
 9   worldwideboxoffice   678 non-null    int64 
 10  firstyear            678 non-null    int64 
 11  last year            678 non-null    int64 
 12  #years               678 non-null    object
dtypes: int64(11), object(2)
memory usage: 69.0+ KB


In [86]:
extfranchises.to_csv('extfranchises_clean.csv')